<a href="https://colab.research.google.com/github/gioalvaro/Coursera_Capstone/blob/master/Cluster_Canada_Neighborhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Introduction
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in Canada. You will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the k-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in Canada and their emerging clusters.
We will start retrieving and processing the data from a Wikipedia URL provided by IBM Coursera. 


## Part 1: Retrieving Data

### Importing Libraries

In [0]:
import numpy as np # biblioteca para manejar datos vectorizados

import pandas as pd # biblioteca para analisis de datos
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # biblioteca para manejar archivos JSON

#!pip install geopy 
from geopy.geocoders import Nominatim # convierte direcciones en valores de longitud y latitud

import requests # bibliotecas para manejar request
from pandas.io.json import json_normalize # transforma JSON en tablas de pandas

# modulos de ploteo
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium 
import folium # biblioteca para renderizar mapas

from urllib.request import urlopen
from bs4 import BeautifulSoup

print('Biblioteca importadas.')

Biblioteca importadas.


### Retrieving from Wikipedia Postal Codes of Canada

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

html = urlopen(url)

In [0]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [0]:
text = soup.get_text()


In [0]:
rows = soup.find_all('tr')
print(rows[:10])

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [0]:
for row in rows:
  row_td = row.find_all('td')
  print(row_td)
  type(row_td)

[]
[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>]
[<td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
</td>]
[<td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>]
[<td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>]
[<td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td>]
[<td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td>]
[<td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heig

In [0]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


str

In [0]:
list_rows = list_rows[1:len(list_rows)-5]

In [0]:
df = pd.DataFrame(list_rows)
df.head(10)

,0
0,"[M1A, Not assigned, Not assigned\n]"
1,"[M2A, Not assigned, Not assigned\n]"
2,"[M3A, North York, Parkwoods\n]"
3,"[M4A, North York, Victoria Village\n]"
4,"[M5A, Downtown Toronto, Harbourfront\n]"
5,"[M5A, Downtown Toronto, Regent Park\n]"
6,"[M6A, North York, Lawrence Heights\n]"
7,"[M6A, North York, Lawrence Manor\n]"
8,"[M7A, Queen's Park, Not assigned\n]"
9,"[M8A, Not assigned, Not assigned\n]"


### Preprocessing the retrieved data

>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [0]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)
df1.columns = ['PostalCode', 'Borough', 'Neighborhood']
df1.head(10)

,PostalCode,Borough,Neighborhood
0,[M1A,Not assigned,Not assigned\n]
1,[M2A,Not assigned,Not assigned\n]
2,[M3A,North York,Parkwoods\n]
3,[M4A,North York,Victoria Village\n]
4,[M5A,Downtown Toronto,Harbourfront\n]
5,[M5A,Downtown Toronto,Regent Park\n]
6,[M6A,North York,Lawrence Heights\n]
7,[M6A,North York,Lawrence Manor\n]
8,[M7A,Queen's Park,Not assigned\n]
9,[M8A,Not assigned,Not assigned\n]


In [0]:
df1.Borough = df1.Borough.astype('str')

>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [0]:
df_canada = df1[~df1.Borough.str.contains('Not assigned')].copy()

In [0]:
df_canada.PostalCode = df_canada.PostalCode.str.replace('[','')
df_canada.Neighborhood = df_canada.Neighborhood.str.replace('\n]','')

In [0]:
df_canada.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights



> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.



In [0]:
df_canada_grouped = df_canada.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x))

> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [0]:
df_canada_grouped = df_canada_grouped.reset_index()

>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [0]:
df_canada_NotAssigned = df_canada_grouped[df_canada_grouped.Neighborhood.str.contains('Not assigned')]

In [0]:
df_canada_NotAssigned.Neighborhood[:] = df_canada_NotAssigned.Borough[:].copy()


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_with(key, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
df_canada_NotAssigned

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [0]:
replace_values = df_canada_NotAssigned.index.values

In [0]:
df_canada_grouped.iloc[replace_values] = df_canada_NotAssigned

>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [0]:
df_canada_grouped.shape

(103, 3)

## Part 2: Geocoder

### Importing Geocoder

In [0]:
!pip install geocoder

     |████████████████████████████████| 102kB 3.9MB/s 


### Using Geocoder, but if It will not retrieve the coordinates, IBM has just provided in a csv file through an url 

In [0]:
import geocoder # import geocoder

#lat = []
#lon = []
#for postal_code in df_canada_grouped.PostalCode:
#  # initialize your variable to None
#  lat_lng_coords = None
#  # loop until you get the coordinates
#  while(lat_lng_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#    lat_lng_coords = g.latlng

#  latitude = lat_lng_coords[0]
#  longitude = lat_lng_coords[1]
#  lat.append(latitude)
#  lon.append(longitude)

*We cannot make it work so we will use the csv file.*

In [0]:
Geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [0]:
Geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
df_geolocation = df_canada_grouped.merge(Geospatial_data, how = 'inner', left_on = 'PostalCode', right_on = 'Postal Code')

In [0]:
df_geolocation = df_geolocation.drop(['Postal Code'], axis = 1)

In [0]:
df_geolocation.shape

(103, 5)

## Part 3: Cluster Data Analysis

> Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1.   to add enough Markdown cells to explain what you decided to do and to report any observations you make.

2.   to generate maps to visualize your neighborhoods and how they cluster together.



In [0]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_geolocation['Borough'].unique()),
        df_geolocation.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [0]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [0]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geolocation['Latitude'], df_geolocation['Longitude'], df_geolocation['Borough'], df_geolocation['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [0]:
df_geolocation

### Filtering the Borough to keep just the ones that contains the word 'Toronto'

In [0]:
df_geolocation_with_Toronto_Borough = df_geolocation[df_geolocation['Borough'].str.contains('Toronto')]

### Printing the number of Borough and Neighborhoods

In [0]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_geolocation_with_Toronto_Borough['Borough'].unique()),
        df_geolocation_with_Toronto_Borough.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


### Plotting the map with the filtered data

In [0]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geolocation_with_Toronto_Borough['Latitude'], df_geolocation_with_Toronto_Borough['Longitude'], df_geolocation_with_Toronto_Borough['Borough'], df_geolocation_with_Toronto_Borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [50]:
CLIENT_ID = '2FZ10G3CAL4CTAMJMVJWZDUYWUY3AWUELZQTV5TC4GVBVEQS' # your Foursquare ID
CLIENT_SECRET = 'UD3AV5ZN0OPA4OG0HSERVXNSWTASGZQITA0230XSRDQNMCVP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2FZ10G3CAL4CTAMJMVJWZDUYWUY3AWUELZQTV5TC4GVBVEQS
CLIENT_SECRET:UD3AV5ZN0OPA4OG0HSERVXNSWTASGZQITA0230XSRDQNMCVP


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [44]:
df_geolocation_with_Toronto_Borough.iloc[0,2]

' The Beaches'

In [46]:
df_geolocation_with_Toronto_Borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Get the neighborhood's latitude and longitude values.

In [48]:
neighborhood_latitude = df_geolocation_with_Toronto_Borough.iloc[0, 3] # neighborhood latitude value
neighborhood_longitude = df_geolocation_with_Toronto_Borough.iloc[0, 4] # neighborhood longitude value

neighborhood_name = df_geolocation_with_Toronto_Borough.iloc[0, 2] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [0]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the resutls

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d588f534eefd00025d9e314'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [54]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [55]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [57]:
toronto_venues = getNearbyVenues(names=df_geolocation_with_Toronto_Borough['Neighborhood'],
                                   latitudes=df_geolocation_with_Toronto_Borough['Latitude'],
                                   longitudes=df_geolocation_with_Toronto_Borough['Longitude']
                                  )



 The Beaches
 The Danforth West, Riverdale
 The Beaches West, India Bazaar
 Studio District
 Lawrence Park
 Davisville North
 North Toronto West
 Davisville
 Moore Park, Summerhill East
 Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
 Rosedale
 Cabbagetown, St. James Town
 Church and Wellesley
 Harbourfront, Regent Park
 Ryerson, Garden District
 St. James Town
 Berczy Park
 Central Bay Street
 Adelaide, King, Richmond
 Harbourfront East, Toronto Islands, Union Station
 Design Exchange, Toronto Dominion Centre
 Commerce Court, Victoria Hotel
 Roselawn
 Forest Hill North, Forest Hill West
 The Annex, North Midtown, Yorkville
 Harbord, University of Toronto
 Chinatown, Grange Park, Kensington Market
 CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
 Stn A PO Boxes 25 The Esplanade
 First Canadian Place, Underground city
 Christie
 Dovercourt Village, Dufferin
 Little Portugal, Trinity
 Brockton, Exhibition Place

Let's check the size of the resulting dataframe

In [58]:
print(toronto_venues.shape)
toronto_venues.head()

(1688, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each neighborhood

In [78]:
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
 Adelaide, King, Richmond                                                                                      100
 Berczy Park                                                                                                    56
 Brockton, Exhibition Place, Parkdale Village                                                                   21
 Business Reply Mail Processing Centre 969 Eastern                                                              18
 CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara     15
 Cabbagetown, St. James Town                                                                                    45
 Central Bay Street                                                                                             84
 Chinatown, Grange Park, Kensington Market                                                                     100
 Christie                                                          

Let's find out how many unique categories can be curated from all the returned venues

In [60]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


### Analyze Each Neighborhood

In [61]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [62]:
toronto_onehot.shape

(1688, 235)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [63]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.030000,0.0000,0.000000,0.000000,0.0000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.030000,0.000000,0.00000,0.01,0.020000,0.010000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000,0.040000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.010000,0.000000,0.00,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.017857,0.0000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.035714,0.000000,0.017857,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,...,0.035714,0.000000,0.017857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.035714,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.00,0.000000,0.017857,0.000000,0.017857,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.047619,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.095238,0.000000,0.00000,0.00,0.000000,0.047619,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eas...,0.055556,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.055556,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.00000,0.00,0.000000,0.055556,0.000000,...,0.000000,0.000000,0.000000,0.055556,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harb...",0.000000,0.000000,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.000000,

Let's confirm the new size

In [64]:
toronto_grouped.shape

(38, 235)

Let's print each neighborhood along with the top 5 most common venues

In [65]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


---- Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


---- Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0                  Café  0.10
1        Breakfast Spot  0.10
2           Coffee Shop  0.10
3         Grocery Store  0.05
4  Caribbean Restaurant  0.05


---- Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2          Restaurant  0.06
3          Smoke Shop  0.06
4             Brewery  0.06


---- CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Serv

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Asian Restaurant,Gym,American Restaurant,Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop,Café,Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Gym,Burrito Place,Caribbean Restaurant,Stadium,Restaurant,Bar,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eas...,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Recording Studio
4,"CN Tower, Bathurst Quay, Island airport, Harb...",Airport Service,Airport Terminal,Airport Lounge,Boutique,Coffee Shop,Boat or Ferry,Sculpture Garden,Airport Food Court,Airport,Harbor / Marina


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [68]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [69]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_geolocation_with_Toronto_Borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Sushi Restaurant,Pet Store,Gym,Coffee Shop,Pub,Movie Theater,Burrito Place,Burger Joint
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Brewery,Cheese Shop,Fish Market,Latin American Restaurant,Seafood Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Finally, let's visualize the resulting clusters

In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Health Food Store,Trail,Pub,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Restaurant
42,East Toronto,0,Park,Sandwich Place,Sushi Restaurant,Pet Store,Gym,Coffee Shop,Pub,Movie Theater,Burrito Place,Burger Joint
43,East Toronto,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Brewery,Cheese Shop,Fish Market,Latin American Restaurant,Seafood Restaurant
45,Central Toronto,0,Breakfast Spot,Clothing Store,Sandwich Place,Gym,Park,Hotel,Food & Drink Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
46,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Yoga Studio,Mexican Restaurant,Restaurant,Rental Car Location,Salon / Barbershop,Diner,Furniture / Home Store,Chinese Restaurant
47,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Café,Italian Restaurant,Coffee Shop,Sushi Restaurant,Greek Restaurant,Seafood Restaurant,Restaurant
49,Central Toronto,0,Pub,Light Rail Station,Coffee Shop,Pizza Place,Sports Bar,Restaurant,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint
51,Downtown Toronto,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Bakery,Pizza Place,Restaurant,Chinese Restaurant,Breakfast Spot
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Mediterranean Restaurant,Fast Food Restaurant,Pub,Gym,Bubble Tea Shop


#### Cluster 2

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Playground,Tennis Court,Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 3

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Women's Store,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 4

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,3,Park,Playground,Trail,Building,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 5

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Bus Line,Swim School,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [76]:
toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]

Index(['Borough', 'Cluster Labels', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

In [77]:
toronto_merged.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')